In [2]:
import sys
import ee
import geemap

try:
    # Initialize the library.
    ee.Authenticate()
    ee.Initialize()
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise
    
    


Google Earth Engine has initialized successfully!


In [15]:
def add_metrics(image):
    ndvi = image.normalizedDifference(['B7', 'B4']).rename('NDVI')
    gndvi = image.normalizedDifference(['B7', 'B3']).rename('GNDVI')   
    savi = ee.Image().expression('((NIR-Red)/(NIR+Red+0.5))*1.5', {
        'NIR': image.select('B8').multiply(0.0001),
        'Red': image.select('B4').multiply(0.0001)
      }).rename('SAVI')
    return image.addBands([ndvi, gndvi, savi])

def create_gif_from_collection(image_collection,
                               clipRegion,
                               bands,
                               min_val,
                               max_val, 
                               palette: ['#640000', '#ff0000', '#ffff00', '#00c800', '#006400']):
    filmArgs = {
    "dimensions": 128,
    "region": clipRegion,
    "crs": 'EPSG:3857',
    "min": min_val, 
    "max" : max_val,
    "palette":  palette,
    "bands": bands,
    "framesPerSecond": 1, 
}
    return image_collection.map(add_metrics).filterDate('2019-02-01', '2019-06-30').filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 25).getVideoThumbURL(filmArgs)

def listOfImages_printer(imagesList):
    print(len(imagesList))
    for i in range(len(imagesList)):
        if i < 10:
            print(i,"    ",imagesList[i])
        else: 
            print(i,"   ",imagesList[i])

        if (i+1) % 3 == 0:  
            print(" ")
            
def get_image_date(image): 
    return ee.Date(ee.Image(image).get('system:time_start')).format("yyyy-MM-dd").getInfo()

def show_image(Map, ImagesCollection, index, clipRegion, params):
    Images = ImagesCollection.toList(ImagesCollection.size())
    Image = ee.Image(Images.get(index)).clip(clipRegion)
    Image_metrics = add_metrics(Image).clip(clipRegion)
    print(get_image_date(Image_metrics))
    Map.addLayer(Image_metrics, params, "", True) 
    return Map

def show_image_diff(Map, ImagesCollection, index_1, index_2, clipRegion, params):
    Images = ImagesCollection.toList(ImagesCollection.size())
    Image_1 = ee.Image(Images.get(index_1)).clip(clipRegion)
    Image_1_metrics = add_metrics(Image_1)

    Image_2 = ee.Image(Images.get(index_2)).clip(clipRegion)
    Image_2_metrics = add_metrics(Image_2)
    print(get_image_date(Image_2), get_image_date(Image_1))
    Image_diff = (Image_1_metrics.subtract(Image_2_metrics)).abs()
    Map.addLayer(Image_diff, params, "", True) 
    return Map

def get_image(ImagesCollection, index, clipRegion):
    Images = ImagesCollection.toList(ImagesCollection.size())
    Image = ee.Image(Images.get(index)).clip(clipRegion)
    return ee.Image(add_metrics(Image).clip(clipRegion))


In [6]:
region = ee.Geometry.Polygon([[[6.91195, 52.188441],
  [6.898218, 52.185283],
  [6.891522, 52.187388],
  [6.887917, 52.182336],
  [6.879504, 52.180968],
  [6.864739, 52.182021],
  [6.867315, 52.186757],
  [6.857185, 52.186231],
  [6.847743, 52.189177],
  [6.843794, 52.177705],
  [6.839502, 52.172231],
  [6.831261, 52.171915],
  [6.828342, 52.167598],
  [6.831776, 52.160017],
  [6.828858, 52.152539],
  [6.846369, 52.150328],
  [6.864053, 52.15096],
  [6.859074, 52.156015],
  [6.854782, 52.165071],
  [6.859074, 52.169178],
  [6.865255, 52.162228],
  [6.873152, 52.161596],
  [6.877444, 52.157911],
  [6.903537, 52.169809],
  [6.904739, 52.175284],
  [6.910748, 52.178231],
  [6.91607, 52.176863],
  [6.917444, 52.180547],
  [6.926886, 52.181284],
  [6.927916, 52.179179],
  [6.940793, 52.180337],
  [6.940964, 52.184336],
  [6.943883, 52.189388],
  [6.954871, 52.193492],
  [6.960536, 52.193176],
  [6.969292, 52.202646],
  [6.963111, 52.203067],
  [6.961738, 52.207169],
  [6.951094, 52.206538],
  [6.939419, 52.212534],
  [6.939076, 52.216846],
  [6.931865, 52.216531],
  [6.93238, 52.212008],
  [6.924826, 52.213901],
  [6.917272, 52.214638],
  [6.918645, 52.206538],
  [6.911606, 52.206328],
  [6.912808, 52.194018],
  [6.91195, 52.188441]]]
)

In [7]:
myCollection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(region) \
    .filterDate('2019-02-01', '2019-06-30') \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 25)
listOfImages = myCollection.aggregate_array('system:index').getInfo()
print('Number of images in the collection: ', len(listOfImages))

Images = myCollection.toList(myCollection.size())

Number of images in the collection:  27


In [8]:
print(create_gif_from_collection(myCollection))

TypeError: create_gif_from_collection() missing 5 required positional arguments: 'clipRegion', 'bands', 'min_val', 'max_val', and 'palette'

In [8]:
visParams = {"min": 0, 
              "max": 1500, 
              "bands": ["B4", "B3", "B2"]}

ndviParams_diff = {min: 0, max: 1, "palette":  ['00cd00', 'ffff00', 'ffa500', 'cd0000', '8b0000'], "bands": ['NDVI']};

gndviParams_diff = {min: 0, max: 1, "palette":  ['00cd00', 'ffff00', 'ffa500', 'cd0000', '8b0000'], "bands": ['GNDVI']};

saviParams_diff = {min: 0, max: 1, "palette":  ['00cd00', 'ffff00', 'ffa500', 'cd0000', '8b0000'], "bands": ['SAVI']};

ndviParams = {min: -1, max: 1,     "palette":  ['#640000', '#ff0000', '#ffff00', '#00c800', '#006400'], "bands": ['NDVI']};


In [9]:
index = 12

Image_1 = ee.Image(Images.get(index)).clip(region)
Image_1_metrics = add_metrics(Image_1)
print(listOfImages[index])
Map3 = geemap.Map(location=[52.180988, 6.88958], zoom=13)
Map3.addLayer(Image_1_metrics, ndviParams, "MedianImage", True) 
Map3


20190418T104029_20190418T104115_T31UGT


Map(center=[52.180988, 6.88958], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

In [10]:
Map4 = geemap.Map(location=[52.180988, 6.88958], zoom=13)
show_image_diff(Map4, myCollection, 15, 12, region, ndviParams_diff)


20190319T104019_20190319T104101_T31UGT 20190329T104029_20190329T104024_T31UGT


Map(center=[52.180988, 6.88958], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

In [40]:
fc = ee.FeatureCollection(Map4.draw_last_feature)
cords = fc.geometry().getInfo()['coordinates']
cords

[[[6.835697, 52.156778],
  [6.834602, 52.155765],
  [6.836791, 52.154304],
  [6.837456, 52.155594],
  [6.839516, 52.154856],
  [6.839902, 52.155344],
  [6.835697, 52.156778]]]

In [46]:
region_enschede_kersdijk_084_184_2019 = ee.Geometry.Polygon([[[6.95595, 52.204923],
  [6.952894, 52.205837],
  [6.9524, 52.205147],
  [6.954159, 52.20445],
  [6.954395, 52.203509],
  [6.954813, 52.203503],
  [6.95595, 52.204923]]])

region_enschede_berktepaalweg_184_135_2019 = ee.Geometry.Polygon([[[6.835697, 52.156778],
  [6.834602, 52.155765],
  [6.836791, 52.154304],
  [6.837456, 52.155594],
  [6.839516, 52.154856],
  [6.839902, 52.155344],
  [6.835697, 52.156778]]])





In [47]:
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(region_enschede_berktepaalweg_184_135_2019) \
    .filterDate('2019-03-01', '2019-06-30') \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 25)
listOfImages = collection.aggregate_array('system:index').getInfo()
listOfImages_printer(listOfImages)

16
0      20190319T104019_20190319T104101_T31UGT
1      20190319T104019_20190319T104101_T32ULC
2      20190329T104029_20190329T104024_T31UGT
 
3      20190329T104029_20190329T104024_T32ULC
4      20190408T104029_20190408T104058_T31UGT
5      20190408T104029_20190408T104058_T32ULC
 
6      20190418T104029_20190418T104115_T31UGT
7      20190418T104029_20190418T104115_T32ULC
8      20190513T104031_20190513T104026_T31UGT
 
9      20190513T104031_20190513T104026_T32ULC
10     20190607T104029_20190607T104533_T31UGT
11     20190607T104029_20190607T104533_T32ULC
 
12     20190617T104029_20190617T104030_T31UGT
13     20190617T104029_20190617T104030_T32ULC
14     20190627T104029_20190627T104030_T31UGT
 
15     20190627T104029_20190627T104030_T32ULC


In [48]:
Map = geemap.Map(location=[52.205837, 6.952894], zoom=15)
show_image(Map, collection, 6, region_enschede_berktepaalweg_184_135_2019, ndviParams)
# show_image_diff(Map, collection, 18, 21, region_usselo_234_135_2019_glyph, ndviParams_diff)

2019-04-18


Map(center=[52.205837, 6.952894], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [36]:
img_list = [3,4,5]

for i in img_list:
    image = get_image(collection, i, region_enschede_berktepaalweg_184_135_2019)
    field = "Berktepaalweg/"
    filename = "Enschede-South/" + field + get_image_date(image) + ".tif"
    geemap.ee_export_image(
        image,
        filename=filename, 
        scale=30, 
        region=region_enschede_berktepaalweg_184_135_2019, 
        file_per_band=False
)

Generating URL ...
Please wait ...
Data downloaded to D:\Uni\Mod_14_Data_Science\assignmentsDS_Git\ComputerVisionAssignment\ComputerVisionAssignment\GIS-project\Enschede-South\Kersdijk\2019-06-17.tif


##### 
